# Make Volumetric Graphs of Delay

In [2]:
colors_base = {
"00":"c64_black",  
"01":"c64_white",  
"02":"c64_red",    
"03":"c64_cyan",   
"04":"c64_purple", 
"05":"c64_green",  
"06":"c64_blue",   
"07":"c64_yellow", 
"08":"c64_orange", 
"09":"c64_brown",  
"0a":"c64_ltred",  
"0b":"c64_gray1",  
"0c":"c64_lightgray",  
"0d":"c64_ltgreen",
"0e":"c64_ltblue", 
"0f":"c64_gray3",  
}
colors = [v for k,v in colors_base.items()]
    
c64_to_rgb = {
"c64_black": "#000000",     
"c64_white": "#ffffff",     
"c64_red": "#880000",       
"c64_cyan":  "#aaffee",     
"c64_purple": "#cc44cc",    
"c64_green": "#00cc55",     
#"c64_blue":  "#0000aa",     
"c64_blue":  "#0088ff",     
"c64_yellow":  "#eeee77",   
"c64_orange":  "#dd8855",   
"c64_brown": "#664400",     
"c64_ltred": "#ff7777",  
"c64_gray1":  "#333333",    
"c64_ltgreen": "#aaff66",
"c64_ltblue":  "#0088ff",
"c64_lightgrey": "#bbbbbb", 
"c64_lightgray": "#bbbbbb", 
"c64_gray3": "#bbbbbb",                             
}


Run the following command:
```sh
~/Dev/vice-emu-code/vice/src/x64sc -directory ~/Dev/vice-emu-code/vice/data/ -moncommands painttrace.txt bin/psychedelia.prg 
```
Where `painttrace.txt` is:

```
log on
logname "PsychedeliaIndividualPatterns.txt"
tr exec 0973
command 1 "mem d800 dbff"
```

Read in the ram history.

In [72]:
log_file = "PsychedeliaDelayPaintTrace.txt"
input_file = open(log_file,'r')

ram_history = []
lines = input_file.readlines()
c = 0
for i in range(5, len(lines), 67):
    raw_ram = [l[9:60].split() for l in lines[i:i+64]]
    raw_ram = [int(v,16) & 0x0f for l in raw_ram for v in l]
    color_ram = list(reversed([raw_ram[v:v+40] for v in range(0, len(raw_ram), 40)]))[2:]
    ram_history += [color_ram]
    c += 1


Slice up the ram history into the different patterns.

In [186]:
flatten = lambda l: [e for sublist in l for e in sublist]

pattern_breaks = []
for i,h in enumerate(ram_history):
    num_pixels = sum(flatten(h))
    if num_pixels > 1:
        continue
    pattern_breaks += [i]

npb = []
pp = 0
MAX_GAP = 15
for p in pattern_breaks:
    if p - pp < MAX_GAP:
        pp = p
        continue
    npb += [pp]
    pp = p
npb += [pp]

patterns = []
pp = -1
for p in npb:
    patterns += [ram_history[pp:p]]
    pp = p
patterns = patterns[1:]

In [187]:
[len(p) for p in patterns]

[279, 224]

In [189]:
from PIL import Image, ImageColor
import os

def aggregatePattern(patt):
    agg_pattern = [[[] for i in range(40)] for i in range(24)]
    for p in patt:
        for r,row in enumerate(p):
            for c,col in enumerate(row):
                if col:
                    agg_pattern[r][c] += [col]
    return agg_pattern

for pn, patt in enumerate(patterns):
    no_pixels = len(patt)
    agg_pattern = aggregatePattern(patt)
    cell_hw = math.ceil(math.sqrt(no_pixels)) 
    print(cell_hw)
    screen_rows = 40
    screen_cols = 24

    img = Image.new( 'RGB', ((screen_rows*cell_hw),
                             (screen_cols*cell_hw)), "black")
    pixels = img.load()
    for r,row in enumerate(agg_pattern):
        y_o = (r * cell_hw) + 1
        for c,cell_pixels in enumerate(row):
            x_o = (c * cell_hw) + 1
            for i,px in enumerate(cell_pixels):
                x = i % (cell_hw - 1)
                y = math.floor((i / (cell_hw))) 
                color = ImageColor.getrgb(c64_to_rgb[colors[px]])
                pixels[x_o+x, y_o+y] = color
    img = img.resize((img.width * 4, img.height * 4), Image.NEAREST)
    img.save(f"delay/pixelhist-{pn}.png")

17
15
